# Stimulus Generation

## Import Libraries

In [5]:
import numpy as np
import moviepy.editor as e
import numpy.random as npr
import time
import os
import json

## Color Screen
This section generates videos of transitioning colors.

In [6]:
def colorScreen(dur=10,h=500,w=500,speed=1,fps=20):
    # number of transitions
    num_switches = int(dur*speed)+1
    # number of frames between transitions
    num_frames = int(fps/speed)
    # begin and end colors
    main_colors = 255*npr.rand(num_switches,3)
    
    colors = [] # labels
    imgs = [] # frames
    
    # compute middle frames by linear interpolation
    for i in range(num_switches-1):
        c1 = main_colors[i] # first color
        c2 = main_colors[i+1] # second color
        for a in np.linspace(0,1,num_frames, endpoint=False):
            interp_color = np.array(c1*(1-a)+c2*a, dtype = np.uint8) # interpolate
            colors.append(interp_color) # append labels
            x = np.ones((h, w, 3)) # initialize the frame
            x[:, :, 0:3] = interp_color
            imgs.append(x)
    return imgs,colors        

In [7]:
def color_stim_generator(folder,**kvargs):
    # kvargs include dur, h, w, speed, fps
    # unpack parameters
    kvargs['experiment'] = 'COLOR'
    dur = 10 if 'dur'not in kvargs else kvargs['dur']
    kvargs['dur'] = dur
    h = 500 if 'h' not in kvargs else kvargs['h']
    kvargs['h'] = h
    w = 500 if 'w' not in kvargs else kvargs['w']
    kvargs['w'] = w
    speed = 1 if 'speed' not in kvargs else kvargs['speed']
    kvargs['speed'] = speed
    fps = 20 if 'fps' not in kvargs else kvargs['fps']
    kvargs['fps'] = fps
    
    # generate frames
    image_list,color_list = colorScreen(dur,h,w,speed,fps)
    # generate unique name
    timestr = time.strftime("%Y%m%d-%H%M%S")
    folder_path = folder+timestr
    os.mkdir(folder_path)
    # make video clip
    clip = e.ImageSequenceClip(image_list,fps)
    # write video file
    clip.write_videofile(folder_path+'/vid_'+timestr+'.mp4', audio=False)
    # write the log file
    with open(folder_path+'/log_'+'.json','w') as jsf:
        kvargs['colors'] = [[int(color) for color in c] for c in color_list] #JSON hates np arrays
        json.dump(kvargs,jsf)
    print('done!')

### Example usecases

In [ ]:
# specify the path to save the files
color_stim_path = '/Users/neo/newprotocol/color_stim/'
# example parameters
color_gen_params = {'dur':10,'h':500,'w':500,'speed':0.5, 'fps':20}

In [85]:
# example with given parameters
color_stim_generator(color_stim_path,**color_gen_params)

t:  15%|█▌        | 31/201 [00:00<00:00, 308.09it/s, now=None]

Moviepy - Building video /Users/neo/newprotocol/color_stim/20190603-172522/vid_20190603-172522.mp4.
Moviepy - Writing video /Users/neo/newprotocol/color_stim/20190603-172522/vid_20190603-172522.mp4



Moviepy - Done !
Moviepy - video ready /Users/neo/newprotocol/color_stim/20190603-172522/vid_20190603-172522.mp4
done!


In [87]:
# example with default parameters and two user specified params
color_stim_generator(color_stim_path, fps=30, h=800)

t:   7%|▋         | 21/300 [00:00<00:01, 199.67it/s, now=None]

Moviepy - Building video /Users/neo/newprotocol/color_stim/20190603-172551/vid_20190603-172551.mp4.
Moviepy - Writing video /Users/neo/newprotocol/color_stim/20190603-172551/vid_20190603-172551.mp4



Moviepy - Done !
Moviepy - video ready /Users/neo/newprotocol/color_stim/20190603-172551/vid_20190603-172551.mp4
done!


## Cursor Screen
This section generates videos of a moving white square (cursor) on black background

In [8]:
def cursorScreen(dur=10,h=500,w=500,speed=1,fps=20, cursor_size = 10):
    # number of transitions
    num_switches = int(dur*speed)+1
    # number of frames between transitions
    num_frames = int(fps/speed)
    
    # check if cursor size is smaller than the screen size
    if (cursor_size >=w or cursor_size >=h):
        raise Exception('cursor size too big')
        
    # generate the x coordinates and make sure the cursor won't fall out of the borders
    x_coords = (w-cursor_size)*npr.rand(num_switches)
    # generate the y coordinates and make sure the cursor won't fall out of the borders
    y_coords = (h-cursor_size)*npr.rand(num_switches)
    
    imgs = []
    positions = [] #labels
    
    for i in range(num_switches-1):
        # initial position of the cursor
        x1 = x_coords[i]
        y1 = y_coords[i]
        # next position of the cursor
        x2 = x_coords[i+1]
        y2 = y_coords[i+1]
        
        for a in np.linspace(0,1,num_frames, endpoint=False):
            interp_x = np.array(x1*(1-a)+x2*a, dtype = np.int)
            interp_y = np.array(y1*(1-a)+y2*a, dtype = np.int)
            positions.append([interp_x,interp_y])
            img = np.zeros((h, w, 3))
            img[interp_y:interp_y+cursor_size, interp_x:interp_x+cursor_size, :] =255 
            imgs.append(img)
    return imgs,positions

In [9]:
def cursor_stim_generator(folder,**kvargs):
    # kvargs include dur, h, w, speed, fps
    # unpack parameters
    kvargs['experiment'] = 'CURSOR'
    dur = 10 if 'dur'not in kvargs else kvargs['dur']
    kvargs['dur'] = dur
    h = 500 if 'h' not in kvargs else kvargs['h']
    kvargs['h'] = h
    w = 500 if 'w' not in kvargs else kvargs['w']
    kvargs['w'] = w
    speed = 1 if 'speed' not in kvargs else kvargs['speed']
    kvargs['speed'] = speed
    fps = 20 if 'fps' not in kvargs else kvargs['fps']
    kvargs['fps'] = fps
    cursor_size = 10 if 'cursor_size' not in kvargs else kvargs['cursor_size']
    kvargs['cursor_size'] = cursor_size
    
    # generate frames
    image_list,pos_list = cursorScreen(dur,h,w,speed,fps,cursor_size)
    # generate unique name
    timestr = time.strftime("%Y%m%d-%H%M%S")
    folder_path = folder+timestr
    os.mkdir(folder_path)
    # make video clip
    clip = e.ImageSequenceClip(image_list,fps)
    # write video file
    clip.write_videofile(folder_path+'/vid_'+timestr+'.mp4', audio=False)
    # write the log file
    with open(folder_path+'/log_'+'.json','w') as jsf:
        kvargs['positions'] = [[int(pos) for pos in p] for p in pos_list] #JSON hates np arrays
        json.dump(kvargs,jsf)
    print('done!')

### Example Usecases

In [ ]:
cursor_stim_path = '/Users/neo/newprotocol/cursor_stim/'
cursor_gen_params = {'dur':10,'h':500,'w':500,'speed':0.5, 'fps':20, 'cursor_size':10}

In [115]:
cursor_stim_generator(cursor_stim_path,**cursor_gen_params)

t:  14%|█▍        | 29/201 [00:00<00:00, 285.41it/s, now=None]

Moviepy - Building video /Users/neo/newprotocol/cursor_stim/20190603-183802/vid_20190603-183802.mp4.
Moviepy - Writing video /Users/neo/newprotocol/cursor_stim/20190603-183802/vid_20190603-183802.mp4



Moviepy - Done !
Moviepy - video ready /Users/neo/newprotocol/cursor_stim/20190603-183802/vid_20190603-183802.mp4
done!


## Color_Cursor Screen
This section generates videos of a moving white square (cursor) on colored background where the color changes with a potentially different speed

In [10]:
def color_cursor_screen(dur=10, h=500, w=500,
                        color_speed=0.5, cursor_speed=1,
                        cursor_size=15, fps=20):
    cursor_imgs,pos_list = cursorScreen(dur=dur,h=h
                                        ,w=w,speed=cursor_speed,
                                        fps=fps, cursor_size = cursor_size)
    color_imgs,color_list = colorScreen(dur=dur,h=h,w=w,speed=color_speed,fps=fps)
    all_imgs = [np.maximum(cursor_imgs[i],color_imgs[i]) for i in range(len(cursor_imgs))]
    return all_imgs,pos_list,color_list


In [11]:
def color_cursor_stim_generator(folder,**kvargs):
    # kvargs include dur, h, w, speed, fps
    # unpack parameters
    kvargs['experiment'] = 'COLOR_CURSOR'
    dur = 10 if 'dur'not in kvargs else kvargs['dur']
    kvargs['dur'] = dur
    h = 500 if 'h' not in kvargs else kvargs['h']
    kvargs['h'] = h
    w = 500 if 'w' not in kvargs else kvargs['w']
    kvargs['w'] = w
    color_speed = 1 if 'color_speed' not in kvargs else kvargs['color_speed']
    kvargs['color_speed'] = color_speed
    cursor_speed = 1 if 'cursor_speed' not in kvargs else kvargs['cursor_speed']
    kvargs['cursor_speed'] = cursor_speed
    fps = 20 if 'fps' not in kvargs else kvargs['fps']
    kvargs['fps'] = fps
    cursor_size = 10 if 'cursor_size' not in kvargs else kvargs['cursor_size']
    kvargs['cursor_size'] = cursor_size
    
    # generate frames
    image_list,pos_list,color_list = color_cursor_screen(dur,h,w,color_speed,cursor_speed,cursor_size,fps)
    # generate unique name
    timestr = time.strftime("%Y%m%d-%H%M%S")
    folder_path = folder+timestr
    os.mkdir(folder_path)
    # make video clip
    clip = e.ImageSequenceClip(image_list,fps)
    # write video file
    clip.write_videofile(folder_path+'/vid_'+timestr+'.mp4', audio=False)
    # write the log file
    with open(folder_path+'/log_'+'.json','w') as jsf:
        kvargs['colors'] = [[int(color) for color in c] for c in color_list] #JSON hates np arrays
        kvargs['positions'] = [[int(pos) for pos in p] for p in pos_list] #JSON hates np arrays
        json.dump(kvargs,jsf)
    print('done!')

### Example Usecases

In [155]:
color_cursor_stim_path = '/Users/neo/newprotocol/color_cursor_stim/'
color_cursor_gen_params = {'dur':10,'h':500,'w':500,'color_speed':1,'cursor_speed':2, 'fps':60, 'cursor_size':10}


In [156]:
color_cursor_stim_generator(color_cursor_stim_path,**color_cursor_gen_params)

t:   4%|▍         | 26/601 [00:00<00:02, 259.34it/s, now=None]

Moviepy - Building video /Users/neo/newprotocol/color_cursor_stim/20190603-192316/vid_20190603-192316.mp4.
Moviepy - Writing video /Users/neo/newprotocol/color_cursor_stim/20190603-192316/vid_20190603-192316.mp4



Moviepy - Done !
Moviepy - video ready /Users/neo/newprotocol/color_cursor_stim/20190603-192316/vid_20190603-192316.mp4
done!


## Scaling_Cursor Screen
This section generates videos of a moving and scaling white rectangle on a black background

In [32]:
def scalingCursorScreen(dur=10,h=500,w=500,speed=1,fps=20, min_cursor_size = 20, max_cursor_size = 200):
    num_switches = int(dur*speed)+1
    num_frames = int(fps/speed)
    x_scales = [int(s) for s in npr.uniform(min_cursor_size, max_cursor_size,num_switches)]
    y_scales = [int(s) for s in npr.uniform(min_cursor_size, max_cursor_size,num_switches)]
    x_coords = [(w-x)*npr.rand() for x in x_scales]
    y_coords = [(h-y)*npr.rand() for y in y_scales]
    imgs = []
    positions = []
    scales = []
    for i in range(num_switches-1):
        x1 = x_coords[i]
        sx1 = x_scales[i]
        x2 = x_coords[i+1]
        sx2 = x_scales[i+1]
        y1 = y_coords[i]
        sy1 = y_scales[i]
        y2 = y_coords[i+1]
        sy2 = y_scales[i+1]
        for a in np.linspace(0,1,num_frames, endpoint=False):
            interp_x = np.array(x1*(1-a)+x2*a, dtype = np.int)
            interp_y = np.array(y1*(1-a)+y2*a, dtype = np.int)
            s_interp_x = np.array(sx1*(1-a)+sx2*a, dtype = np.int)
            s_interp_y = np.array(sy1*(1-a)+sy2*a, dtype = np.int)
            positions.append([interp_x,interp_y])
            scales.append([s_interp_x,s_interp_y])
            img = np.zeros((h, w, 3))
            img[interp_y:interp_y+s_interp_y, interp_x:interp_x+s_interp_x, :] =255 
            imgs.append(img)
    return imgs,positions,scales

In [33]:
def scaling_cursor_stim_generator(folder,**kvargs):
    # kvargs include dur, h, w, speed, fps, max_cursor_size
    # unpack parameters
    kvargs['experiment'] = 'SCALING_CURSOR'
    dur = 10 if 'dur'not in kvargs else kvargs['dur']
    kvargs['dur'] = dur
    h = 500 if 'h' not in kvargs else kvargs['h']
    kvargs['h'] = h
    w = 500 if 'w' not in kvargs else kvargs['w']
    kvargs['w'] = w
    speed = 1 if 'speed' not in kvargs else kvargs['speed']
    kvargs['speed'] = speed
    fps = 20 if 'fps' not in kvargs else kvargs['fps']
    kvargs['fps'] = fps
    max_cursor_size = 100 if 'max_cursor_size' not in kvargs else kvargs['max_cursor_size']
    kvargs['max_cursor_size'] = max_cursor_size
    min_cursor_size = 20 if 'min_cursor_size' not in kvargs else kvargs['min_cursor_size']
    kvargs['min_cursor_size'] = min_cursor_size
    
    # generate frames
    image_list,pos_list,scale_list = scalingCursorScreen(dur,h,w,speed,fps,min_cursor_size,max_cursor_size)
    # generate unique name
    timestr = time.strftime("%Y%m%d-%H%M%S")
    folder_path = folder+timestr
    os.mkdir(folder_path)
    # make video clip
    clip = e.ImageSequenceClip(image_list,fps)
    # write video file
    clip.write_videofile(folder_path+'/vid_'+timestr+'.mp4', audio=False)
    # write the log file
    with open(folder_path+'/log_'+'.json','w') as jsf:
        kvargs['scales'] = [[int(scale) for scale in s] for s in scale_list] #JSON hates np arrays
        kvargs['positions'] = [[int(pos) for pos in p] for p in pos_list] #JSON hates np arrays
        json.dump(kvargs,jsf)
    print('done!')

### Example Usecases

In [179]:
scaling_cursor_stim_path = '/Users/neo/newprotocol/scaling_cursor_stim/'
scaling_cursor_gen_params = {'dur':20,'h':500,'w':500,'speed':2, 'fps':60,'min_cursor_size':40,'max_cursor_size':200}

In [180]:
scaling_cursor_stim_generator(scaling_cursor_stim_path,**scaling_cursor_gen_params)

t:   3%|▎         | 37/1201 [00:00<00:03, 364.96it/s, now=None]

Moviepy - Building video /Users/neo/newprotocol/scaling_cursor_stim/20190603-205821/vid_20190603-205821.mp4.
Moviepy - Writing video /Users/neo/newprotocol/scaling_cursor_stim/20190603-205821/vid_20190603-205821.mp4



Moviepy - Done !
Moviepy - video ready /Users/neo/newprotocol/scaling_cursor_stim/20190603-205821/vid_20190603-205821.mp4
done!


## Scaling_Color_Cursor Screen
This section generates videos of a moving and scaling colored rectangle on a coloerd background

In [16]:
def scalingColorCursorScreen(dur=10,h=500,w=500,cursor_speed=1,color_speed_in=2,color_speed_out=1,fps=20, min_cursor_size = 20, max_cursor_size = 200):
    cursor_imgs_list,pos_list,scale_list = scalingCursorScreen(dur,h,w,cursor_speed,fps,min_cursor_size,max_cursor_size)
    out_color_imgs,out_color_list = colorScreen(dur=dur,h=h,w=w,speed=color_speed_out,fps=fps)
    in_color_imgs,in_color_list = colorScreen(dur=dur,h=h,w=w,speed=color_speed_in,fps=fps)
    colored_cursor_imgs = [np.multiply(cursor_imgs_list[i]//255,in_color_imgs[i]) for i in range(len(cursor_imgs_list))]
    mask_imgs = [np.multiply(1-(cursor_imgs_list[i]//255),out_color_imgs[i]) for i in range(len(cursor_imgs_list))]
    all_imgs = [colored_cursor_imgs[i]+mask_imgs[i] for i in range(len(cursor_imgs_list))]
    return all_imgs, pos_list, scale_list, in_color_list, out_color_list 


In [17]:
def scaling_color_cursor_stim_generator(folder,**kvargs):
    # kvargs include dur, h, w, speed, fps, max_cursor_size
    # unpack parameters
    kvargs['experiment'] = 'SCALING_CURSOR'
    dur = 10 if 'dur'not in kvargs else kvargs['dur']
    kvargs['dur'] = dur
    h = 500 if 'h' not in kvargs else kvargs['h']
    kvargs['h'] = h
    w = 500 if 'w' not in kvargs else kvargs['w']
    kvargs['w'] = w
    color_speed_in = 2 if 'color_speed_in' not in kvargs else kvargs['color_speed_in']
    kvargs['color_speed_in'] = color_speed_in
    color_speed_out = 1 if 'color_speed_out' not in kvargs else kvargs['color_speed_out']
    kvargs['color_speed_out'] = color_speed_out
    cursor_speed = 1 if 'cursor_speed' not in kvargs else kvargs['cursor_speed']
    kvargs['cursor_speed'] = cursor_speed
    fps = 20 if 'fps' not in kvargs else kvargs['fps']
    kvargs['fps'] = fps
    max_cursor_size = 100 if 'max_cursor_size' not in kvargs else kvargs['max_cursor_size']
    kvargs['max_cursor_size'] = max_cursor_size
    min_cursor_size = 20 if 'min_cursor_size' not in kvargs else kvargs['min_cursor_size']
    kvargs['min_cursor_size'] = min_cursor_size
    
    # generate frames
    image_list,pos_list,scale_list,in_color_list,out_color_list= scalingColorCursorScreen(
        dur,h,w,cursor_speed,color_speed_in,color_speed_out,fps,min_cursor_size,max_cursor_size)
    # generate unique name
    timestr = time.strftime("%Y%m%d-%H%M%S")
    folder_path = folder+timestr
    os.mkdir(folder_path)
    # make video clip
    clip = e.ImageSequenceClip(image_list,fps)
    # write video file
    clip.write_videofile(folder_path+'/vid_'+timestr+'.mp4', audio=False)
    # write the log file
    with open(folder_path+'/log_'+'.json','w') as jsf:
        kvargs['scales'] = [[int(scale) for scale in s] for s in scale_list] #JSON hates np arrays
        kvargs['positions'] = [[int(pos) for pos in p] for p in pos_list] #JSON hates np arrays
        kvargs['in_colors'] = [[int(color) for color in c] for c in in_color_list] #JSON hates np arrays
        kvargs['out_colors'] = [[int(color) for color in c] for c in out_color_list] #JSON hates np arrays
        json.dump(kvargs,jsf)
    print('done!')

In [24]:
scaling_color_cursor_stim_path = '/Users/neo/newprotocol/scaling_color_cursor_stim/'
scaling_color_cursor_gen_params = {'dur':20,'h':500,'w':500,
                                   'cursor_speed':2,'color_speed_in':1,'color_speed_out':2,
                                   'fps':60,'min_cursor_size':20,'max_cursor_size':200}


In [25]:
scaling_color_cursor_stim_generator(scaling_color_cursor_stim_path,**scaling_color_cursor_gen_params)

t:   1%|          | 13/1201 [00:00<00:09, 125.37it/s, now=None]

Moviepy - Building video /Users/neo/newprotocol/scaling_color_cursor_stim/20190604-140036/vid_20190604-140036.mp4.
Moviepy - Writing video /Users/neo/newprotocol/scaling_color_cursor_stim/20190604-140036/vid_20190604-140036.mp4



Moviepy - Done !
Moviepy - video ready /Users/neo/newprotocol/scaling_color_cursor_stim/20190604-140036/vid_20190604-140036.mp4
done!


In [139]:
def message_screen(text,dur=10,fps=20, h=500,w=500):
#     start_audio = AudioFileClip('start.wav')
#     end_audio = AudioFileClip('end.wav')
    img = TextClip(text,color='white',bg_color='black',size=(h,w),fontsize=30)
    img = img.img
    imgs = [img for i in range(fps*dur)]
    return imgs,text

image_list,message = message_screen(text = 'move your head',dur=10,fps=20)
clip = ImageSequenceClip(image_list,fps=20)
clip.write_videofile('message.mp4', audio=False)

In [23]:
cursor_imgs_list,pos_list,scale_list = scalingCursorScreen()

In [26]:
len(cursor_imgs_list)

200

In [30]:
result = np.stack(cursor_imgs_list, axis=0)

In [31]:
result.shape

(200, 500, 500, 3)

In [28]:
cursor_imgs_list[0].shape

(500, 500, 3)